In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import os 
os.chdir('./drive/MyDrive/sharif/MachineLearning/ipython(guide)')

FileNotFoundError: ignored

### Libs

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler
from google.colab import drive
import ast
import re
import pickle


## 1 - Collecting Data


In [6]:
# data = pandas.read_csv('../Data/preprocessed.csv')
data = pandas.read_csv('../Data/preprocessed_50.csv')
print(data['body'])
print(data.shape)

0       اپیدمی کرونا گریبان کسب وکار\u200cها مختلفی جم...
1       نصب اندازی tkinter پست نوشتن برنامه ساده می\u2...
2       تحقیق عوامل مصنوعی اقداماتی دستیابی اهداف انتخ...
3       دیتا ساینس بینی وقوع پدیده شرایط کار مدل اساس ...
4       خیلی\u200cها سوال جواب مثبت میدن واقعیت تلخ ای...
                              ...                        
3022    useRedux میخوام تولز خفن یادم کار\u200cها ریدا...
3023    فرض json rest api میخواید اساس مقادیری فرض jso...
3024    سلام خدمت عزیزان نگاهی خواهیمداشت توسعه برنامه...
3025    آموزش میخوام توضیح بدم چطوری صفحه ارور ۴۰۴ جنگ...
3026    تجزیه تحلیل اطلاعات آگهی\u200cها استخدام کشور ...
Name: body, Length: 3027, dtype: object
(3027, 2)


In [7]:
# freq_dist = pandas.read_csv('../Data/FreqDist_sorted.csv',index_col=False)
freq_dist = pandas.read_csv('../Data/FreqDist_sorted_50.csv',index_col=False)

freq_dist.head()

,word,freq
0,هکتبرفست Hacktoberfest فستیوال جشن ماهه نرم اف...,3
1,جلسه تکمیل مطالب مرتبط Linq Entity Framework خ...,2
2,وقت\u200cها سیستم پیچیده پیاده پیچیدگی سیستم ق...,2
3,میخوام بگم سلام سوال نفر بگن سلام میشه تهه چند...,2
4,مدیر کسب کاری بزرگترین چالش هایش سوال احتمال ن...,2


#### TF-ICF words

In [8]:
# most of words with frequency of 1, are noises or bad detected
freq_dist_slice = freq_dist.sort_values(by='freq',ascending=False)[freq_dist['freq']>2]
print(len(freq_dist_slice))
freq_dist_slice

1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


,word,freq
0,هکتبرفست Hacktoberfest فستیوال جشن ماهه نرم اف...,3


In [9]:
# based on tf_icf we may need only 20% of words
'''
tf_icf_words = freq_dist_slice.iloc[:int(len(freq_dist_slice)*.20)]['word'].values
other_words = freq_dist_slice.iloc[int(len(freq_dist_slice)*.20):]['word'].values
print(len(tf_icf_words))
tf_icf_words[:10]
'''

"\ntf_icf_words = freq_dist_slice.iloc[:int(len(freq_dist_slice)*.20)]['word'].values\nother_words = freq_dist_slice.iloc[int(len(freq_dist_slice)*.20):]['word'].values\nprint(len(tf_icf_words))\ntf_icf_words[:10]\n"

## 2 - Extract Features


#### TF-IDF words

In [12]:
max_features = int(freq_dist_slice.shape[0])
max_features

1

In [13]:
vectorizer = TfidfVectorizer(analyzer='word')#, max_features=max_features
X = vectorizer.fit_transform(data['body'])
print(type(X))
print(X.shape)

<class 'scipy.sparse.csr.csr_matrix'>
(3027, 44048)


In [14]:
# to save
# pickle.dump(vectorizer,open("../Data/tf_idf.pkl","wb"))
pickle.dump(vectorizer,open("../Data/tf_idf_50.pkl","wb"))

# to load 
# tf = pickle.load(open("tfidf1.pkl", 'rb'))
# tf_new = TfidfVectorizer(analyzer='word', ngram_range=(1,2), stop_words = "english", lowercase = True,
#                           max_features = 500000, vocabulary = tf.vocabulary_)

In [15]:
# feature_names = vectorizer.get_feature_names()
X

<3027x44048 sparse matrix of type '<class 'numpy.float64'>'
	with 419909 stored elements in Compressed Sparse Row format>

second way

In [16]:
'''
max_features = int(freq_dist_slice.shape[0]*0.002)
max_features
'''

'\nmax_features = int(freq_dist_slice.shape[0]*0.002)\nmax_features\n'

In [17]:
'''
from sklearn.feature_selection import SelectKBest
vectorizer = TfidfVectorizer(analyzer='word')
df_t = vectorizer.fit_transform(data['body'])
'''

"\nfrom sklearn.feature_selection import SelectKBest\nvectorizer = TfidfVectorizer(analyzer='word')\ndf_t = vectorizer.fit_transform(data['body'])\n"

In [18]:
'''
df_t_reduced = SelectKBest(k=max_features).fit_transform(df_t, data['tag'])
max_features
'''

"\ndf_t_reduced = SelectKBest(k=max_features).fit_transform(df_t, data['tag'])\nmax_features\n"

In [19]:
'''
df_t_reduced.shape
'''

'\ndf_t_reduced.shape\n'

#### SVD

In [29]:
svd_features = int(max_features*0.2)
svd_features , max_features

(0, 1)

In [21]:
X.shape

(3027, 44048)

In [22]:
svd = TruncatedSVD(n_components=3026, random_state=42)
features = svd.fit_transform(X)

In [23]:
features.shape

(3027, 3026)

In [34]:
# to save
# pickle.dump(svd, open("../Data/svd.p","wb"))

pickle.dump(svd, open("../Data/svd_50.p","wb"))

# To load again
# svd = pickle.load(open('../Data/svd.p', 'r'))

## Saving Data

In [31]:
df = pandas.DataFrame(features)
# df.to_csv('../Data/features.csv', index=False)
df.to_csv('../Data/features_50.csv', index=False)
df.shape

(3027, 3026)

In [32]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,2986,2987,2988,2989,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999,3000,3001,3002,3003,3004,3005,3006,3007,3008,3009,3010,3011,3012,3013,3014,3015,3016,3017,3018,3019,3020,3021,3022,3023,3024,3025
0,0.132558,-0.051670,-0.042226,0.040284,-0.006293,-0.022651,0.016667,-0.004053,-0.000983,-0.043292,-0.001102,-0.000843,-0.036757,-0.040195,-0.035983,0.051945,0.014737,-0.031094,-0.002649,0.013659,0.027501,0.022282,-0.020186,-0.028897,0.005587,-0.030627,0.014258,-0.000883,0.010758,0.000814,-0.002982,0.041282,-0.009836,-0.002250,0.005126,0.012491,-0.010215,0.022605,0.003994,0.016821,...,-0.000000e+00,-0.000000e+00,0.000000e+00,0.000000e+00,-0.000000e+00,-0.000000e+00,9.156652e-45,-6.391535e-45,-1.793790e-45,0.000000e+00,-0.000000e+00,-0.000000e+00,-0.000000e+00,-0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000364e-44,-8.839100e-46,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-8.752908e-46,-0.000000e+00,-0.000000e+00,-0.000000e+00,-0.000000e+00,-0.000000e+00,0.000000e+00,-2.018100e-46,-1.492478e-45,-4.190147e-45,-0.000000e+00,0.000000e+00,-0.000000e+00,-3.167273e-45,0.000000e+00,-0.000000e+00,0.000000e+00
1,0.262579,0.156766,0.017497,-0.188504,-0.000931,-0.030321,0.042005,-0.025966,-0.087295,0.043146,-0.063065,-0.004889,0.002481,0.010368,0.088614,0.065151,-0.006403,0.004898,0.110211,-0.123140,0.038186,-0.153943,-0.032753,0.028930,0.066578,-0.040542,-0.025563,-0.027995,-0.051158,-0.009459,-0.030953,0.003733,-0.008698,0.019637,-0.009072,0.003852,-0.001734,0.054567,0.042726,-0.026437,...,-8.775306e-32,-1.859386e-32,5.995120e-32,5.527437e-32,4.827687e-32,9.481912e-32,-1.317689e-32,-3.250038e-32,-5.260566e-33,-3.838067e-32,-1.094634e-32,8.535238e-34,1.487911e-32,-3.460886e-32,3.252419e-33,-3.919375e-33,2.821444e-33,-8.608215e-33,-6.417425e-33,-7.483085e-34,1.238834e-32,1.450469e-33,-3.574620e-33,-2.217638e-32,1.435816e-32,3.522126e-33,1.460167e-32,1.001535e-33,7.392939e-33,-4.131960e-33,1.567850e-32,-7.818510e-33,7.791724e-33,-3.030875e-33,-2.821721e-33,-4.897056e-33,-6.069475e-33,4.656129e-35,1.338243e-33,-4.526180e-35
2,0.245617,-0.090350,-0.085039,-0.030756,-0.094664,0.125780,-0.040997,0.064182,-0.013912,0.020520,-0.000767,-0.011028,-0.039330,0.007868,0.004690,-0.079567,-0.142478,0.131705,0.013784,0.091938,0.004456,-0.035947,0.000831,-0.068191,0.053358,-0.071443,0.075912,0.062205,-0.089248,-0.058942,0.035549,0.001923,-0.018708,0.057952,-0.062931,0.074116,-0.046972,0.003211,-0.006531,0.002785,...,-1.400053e-31,5.053908e-32,3.585815e-32,6.176044e-32,9.036102e-32,-3.650361e-32,-5.641420e-34,-2.100687e-32,3.417637e-33,2.942529e-32,3.914433e-32,-9.398115e-33,3.338891e-32,-6.646057e-33,-9.810594e-34,-8.735894e-34,-2.884500e-34,4.804857e-33,1.161429e-32,-7.028100e-33,1.925588e-32,-3.357941e-33,-9.505924e-33,1.664568e-32,6.407081e-33,1.986072e-32,-1.299269e-32,-8.397926e-34,-3.676462e-33,1.421876e-32,-2.721704e-32,-3.835377e-33,-7.733564e-34,2.586739e-33,2.177543e-33,8.102938e-33,3.225438e-33,1.803109e-33,1.869845e-33,-3.172909e-34
3,0.162129,-0.088841,-0.037773,0.010710,-0.077981,0.126651,0.012601,0.018341,-0.019218,0.016312,-0.019981,-0.042624,-0.068916,-0.038384,-0.019318,-0.066789,-0.096682,0.096520,-0.016320,0.081331,0.032959,-0.083778,-0.044411,-0.084071,0.049944,-0.059877,0.089230,0.072930,-0.021545,-0.073283,0.022345,-0.050764,-0.119145,-0.012199,0.019893,0.041629,-0.045269,0.098892,-0.002375,0.009433,...,-2.556004e-32,-1.713868e-32,2.900229e-32,1.628065e-34,2.007994e-32,5.153266e-32,-3.167758e-32,-4.871277e-33,6.433146e-33,7.919166e-32,8.141656e-35,1.406035e-32,-1.404389e-32,4.765792e-32,-2.103661e-32,5.180421e-33,1.022786e-32,4.862556e-33,-1.805604e-32,2.457341e-33,1.620432e-32,-3.083963e-33,-1.490429e-32,-1.738273e-32,1.069467e-33,-1.104213e-33,2.203283e-32,-8.356078e-34,1.900645e-33,-6.034616e-33,5.119248e-33,-6.449569e-33,-2.229086e-33,-2.296850e-33,-3.769563e-33,-3.773424e-33,-2.569430e-33,-3.957279e-

In [33]:
df.shape

(3027, 3026)